In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# College Insights Dashboard: SQL Queries (02_sql_queries.ipynb)\n",
    "\n",
    "This notebook demonstrates the use of SQL to retrieve key insights from our dataset. We'll leverage a powerful Python library, `sqlite3`, to create an in-memory database from our pandas DataFrame. This approach allows us to write and execute SQL queries without needing a separate database server, which is excellent for a project like this."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Setup and Data Loading\n",
    "\n",
    "First, we'll load the necessary libraries and the cleaned data from our `src/` directory. This ensures we are working with the most up-to-date and prepared dataset."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import sqlite3\n",
    "import logging\n",
    "\n",
    "# Set up logging\n",
    "logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')\n",
    "\n",
    "# Assume we have a function in src/ to load the clean DataFrame\n",
    "import sys\n",
    "sys.path.append('..') # Add the parent directory to the path to import from 'src'\n",
    "from src.load_data import load_all_data\n",
    "\n",
    "# Load the cleaned and merged DataFrame\n",
    "df = load_all_data()\n",
    "\n",
    "if df is not None:\n",
    "    logging.info(\"DataFrame loaded successfully. Creating in-memory database...\")\n",
    "    # Create an in-memory SQLite database connection\n",
    "    conn = sqlite3.connect(':memory:')\n",
    "    \n",
    "    # Write the pandas DataFrame to a SQL table named 'student_data'\n",
    "    df.to_sql('student_data', conn, index=False, if_exists='replace')\n",
    "    \n",
    "    logging.info(\"In-memory database table 'student_data' created.\")\n",
    "    print(\"\\nFirst 5 rows of the SQL table:\\n\")\n",
    "    display(pd.read_sql_query(\"SELECT * FROM student_data LIMIT 5;\", conn))\n",
    "else:\n",
    "    logging.error(\"Could not load data. Please check the `data` directory and `src/load_data.py`.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. SQL Queries for Key Insights\n",
    "\n",
    "Now we will run several advanced SQL queries to answer specific business questions related to college performance. This demonstrates your ability to think critically and translate requirements into database operations."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Query 1: Top 5 Students by Average Marks\n",
    "This query identifies the top-performing students across all subjects, ordered by their average score."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "query_top_students = \"\"\"\n",
    "    SELECT \n",
    "        student_name,\n",
    "        department,\n",
    "        AVG(marks) AS avg_marks\n",
    "    FROM \n",
    "        student_data\n",
    "    GROUP BY \n",
    "        student_name, department\n",
    "    ORDER BY \n",
    "        avg_marks DESC\n",
    "    LIMIT 5;\n",
    "\"\"\"\n",
    "\n",
    "top_students_df = pd.read_sql_query(query_top_students, conn)\n",
    "print(\"Top 5 Students by Average Marks:\")\n",
    "display(top_students_df)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Query 2: Pass Percentage by Subject\n",
    "This query calculates the pass percentage for each subject, which is a key metric for faculty to understand subject difficulty and student performance trends."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "query_pass_percentage = \"\"\"\n",
    "    SELECT \n",
    "        subject_name,\n",
    "        (SUM(CASE WHEN pass_status = 'Pass' THEN 1 ELSE 0 END) * 100.0) / COUNT(*) AS pass_rate_percent\n",
    "    FROM \n",
    "        student_data\n",
    "    GROUP BY \n",
    "        subject_name\n",
    "    ORDER BY \n",
    "        pass_rate_percent DESC;\n",
    "\"\"\"\n",
    "\n",
    "pass_percentage_df = pd.read_sql_query(query_pass_percentage, conn)\n",
    "print(\"Pass Percentage by Subject:\")\n",
    "display(pass_percentage_df)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Query 3: Identify 'At-Risk' Students (Low Attendance)\n",
    "This query pinpoints students with an attendance percentage below a predefined threshold (e.g., 75%), which is a common policy in many institutions. Identifying these students early is critical for intervention."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "query_at_risk_students = \"\"\"\n",
    "    SELECT DISTINCT\n",
    "        student_name, \n",
    "        department, \n",
    "        attendance AS attendance_percentage\n",
    "    FROM \n",
    "        student_data\n",
    "    WHERE \n",
    "        attendance < 75\n",
    "    ORDER BY\n",
    "        attendance_percentage ASC;\n",
    "\"\"\"\n",
    "\n",
    "at_risk_students_df = pd.read_sql_query(query_at_risk_students, conn)\n",
    "print(\"Students with Attendance < 75%:\")\n",
    "display(at_risk_students_df)\n",
    "\n",
    "if at_risk_students_df.empty:\n",
    "    print(\"\\nNo students found with attendance below 75% in the current dataset.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Conclusion\n",
    "\n",
    "This notebook showcases how to leverage SQL for data querying and analysis. The ability to write and execute these queries demonstrates a strong understanding of relational data, a skill that is highly valued by recruiters. The insights gained here will be used to inform our visualizations and dashboard features."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Close the database connection when done\n",
    "if 'conn' in locals() and conn:\n",
    "    conn.close()\n",
    "    logging.info(\"Database connection closed.\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.12"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

: 